In [83]:
import pandas as pd
import os
from datetime import datetime, timedelta
import talib
import pandas as pd
import numpy as np
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
import warnings
import enum
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True) 
import sqlite3
import ipywidgets as widgets
from IPython import display
from IPython.display import clear_output
from tabulate import tabulate

In [84]:
tradingList = {
    "HINDPETRO": {"price": 262.25, "leverage": 10},
    "HINDZINC": {"price": 317.10, "leverage": 10},
    "SBIN": {"price": 364.65, "leverage": 9},
    "ICICIBANK": {"price": 594.75, "leverage": 9},
    "WIPRO": {"price": 507.60, "leverage": 9},
    "ITC": {"price": 203.25, "leverage": 9},
    "BHARTIARTL": {"price": 561.75, "leverage": 9},
    "IOC": {"price": 102.80, "leverage": 9},
    "ONGC": {"price": 115.10, "leverage": 9},
    "DABUR": {"price": 529.20, "leverage": 9},
    "GAIL": {"price": 161.05, "leverage": 9},
    "TATACONSUM": {"price": 640.05, "leverage": 9},
    "AMBUJACEM": {"price": 309.05, "leverage": 9},
    "DLF": {"price": 265.30, "leverage": 8},
    "TATAPOWER": {"price": 109.35, "leverage": 8},
    "TATAMOTORS": {"price": 315.55, "leverage": 7}
}

tickerList = list(tradingList.keys())


In [85]:
class Trend(enum.Enum):
    strong_uptrend = 1
    uptrend = 2
    strong_downtrend = 3
    downtrend = 4
    uptrend_reversal = 5
    downtrend_reversal = 6
    strong_uptrend_reversal = 7
    strong_downtrend_reversal = 8
    sidetrend = 9

In [86]:
class Candle(enum.Enum):
    red = 1
    green = 2
    doji = 3

In [87]:
class CandleHelper:

    def __init__(self, count=4):
        self.count = count
        self.index = 0

    def is_doji_candle(self, candle):
        return candle == Candle.doji

    def is_green_candle(self, candle):
        return candle == Candle.green

    def is_red_candle(self, candle):
        return candle == Candle.red

    def get_last_candle(self, data):
        if data['close'].values > data['open'].values:
            return Candle.green
        if data['close'].values < data['open'].values:
            return Candle.red
        return Candle.doji

    def get_candle_details(self, data):
        candle_data = data.tail(4)
        current = self.get_last_candle(candle_data.tail(1))
        previous = self.get_last_candle(candle_data.tail(2).head(1))
        last_two = self.get_last_candle(candle_data.tail(3).head(1))
        last_three = self.get_last_candle(candle_data.head(1))
        return current, previous, last_two, last_three

    def trend_strength(self, data, trend):
        candle_data = data.tail(4)
        current = candle_data.tail(1)
        previous = candle_data.tail(2).head(1)
        last_two = candle_data.tail(3).head(1)
        last_three = candle_data.head(1)
        reversal_value = 0
        if trend == Trend.strong_uptrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.downtrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.downtrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_uptrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        else:
            trend_value = 0
            reversal_value = 0
        return trend_value, reversal_value

    def get_trends(self, data):
        current, previous, last_two, last_three = self.get_candle_details(data)
        trend = Trend.sidetrend
        if (self.is_green_candle(current) or self.is_doji_candle(current)) and (
                self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend
                else:
                    trend = Trend.uptrend
        elif (self.is_red_candle(current) or self.is_doji_candle(current)) and (
                self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend
                else:
                    trend = Trend.downtrend
        elif self.is_red_candle(current) and (self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend_reversal
                else:
                    trend = Trend.uptrend_reversal
        elif self.is_green_candle(current) and (self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend_reversal
                else:
                    trend = Trend.downtrend_reversal
        else:
            trend = Trend.sidetrend
        trend_value, reversal_value = self.trend_strength(data, trend)
        return trend, trend_value, reversal_value

    def get_next_candle(self,data):
        candle_data = data.tail(1)
        self.index +=1
        return self.get_last_candle(candle_data)

In [88]:
def candle_graph(data,ticker):
    Candle = go.Candlestick(x=data.index,
                       open=data.open,
                       high=data.high,
                       low=data.low,
                       close=data.close
                       )  
    
                   
    iplot([Candle])

In [89]:
class PricePattern(enum.Enum):
    BullishHammer = 1
    BullishHarami = 2
    BullishInverseHammer = 3
    BullishMorningStar = 4
    BullishPiercingLine = 5
    BullishThreeWhiteSoldiers = 6
    BullishEngulfing = 7    
    BearishDarkCloudCover = 8
    BearishEveningStar = 9
    BearishHangingMan = 10
    BearishHarami = 11
    BearishShootingStar = 12
    BearishThreeBlackCrows = 13
    BearishEngulfing = 14

In [90]:
class Graph:
    def __init__(self,pattern=PricePattern.BullishHammer):
        self.pattern = pattern
        self.value = 100
        
    def get_pattern(self,row):
        
        if self.pattern == PricePattern.BullishHammer:
            self.value = 100
            return row["Hammer"]
        
        if self.pattern == PricePattern.BullishHarami:
            self.value = 100
            return row["Harami"]
        
        if self.pattern == PricePattern.BullishInverseHammer:
            self.value = 100
            return row["InverseHammer"]
        
        if self.pattern == PricePattern.BullishMorningStar:
            self.value = 100
            return row["MorningStar"]
        
        if self.pattern == PricePattern.BullishPiercingLine:
            self.value = 100
            return row["PiercingLine"]
        
        if self.pattern == PricePattern.BullishThreeWhiteSoldiers:
            self.value = 100
            return row["WhiteSoldiers"]
        
        if self.pattern == PricePattern.BullishEngulfing:
            self.value = 100
            return row["Engulfing"]
        
        if self.pattern == PricePattern.BearishDarkCloudCover:
            self.value = -100
            return row["DarkCloudCover"]
        
        if self.pattern == PricePattern.BearishEveningStar:
            self.value = -100
            return row["EveningStar"]
        
        if self.pattern == PricePattern.BearishHangingMan:
            self.value = -100
            return row["HangingMan"]
        
        if self.pattern == PricePattern.BearishHarami:
            self.value = -100
            return row["Harami"]
        
        if self.pattern == PricePattern.BearishShootingStar:
            self.value = -100
            return row["ShootingStar"]
        
        if self.pattern == PricePattern.BearishThreeBlackCrows:
            self.value = -100
            return row["ThreeBlackCrows"]
        
        if self.pattern == PricePattern.BearishEngulfing:
            self.value = -100
            return row["Engulfing"]
        
    def show(self,data,ticker):    
        pattern_data = pd.DataFrame(columns=['x','Marker','Symbol','Color'])   
        start = pd.DataFrame(columns=['x','Marker','Symbol','Color'])
        end = pd.DataFrame(columns=['x','Marker','Symbol','Color'])
        helper = CandleHelper()    
        next_long_pattern = False
        trade_open = False
        current_price = 0
        for index, row in data.iterrows():
            start_data = {} 
            end_data = {} 
            if trade_open:
                candle_data = data.loc[index:index]
                closing_price = candle_data['close'].values[0] 
                spread = round(abs(candle_data['open'].values[0]  - closing_price),2)
                pattern = helper.get_next_candle(candle_data)
                if (pattern == Candle.red and spread >= 0.15 ) or closing_price <= current_price - 1:
                    end_data["x"] = index
                    end_data["Marker"] = row["close"]
                    end_data["Symbol"] ="star-triangle-down"
                    end_data["Color"] ="red" 
                    end = end.append(end_data, ignore_index=True) 
                    trade_open = False
            if next_long_pattern:
                if helper.index > 3:
                    helper.index = 0
                    next_long_pattern = False            
                else:
                    pattern = helper.get_next_candle(data.loc[index:index])
                    if pattern == Candle.green:
                        helper.index = 0
                        next_long_pattern = False
                        start_data["x"] = index
                        start_data["Marker"] = row["close"]
                        start_data["Symbol"] ="star-triangle-up"
                        start_data["Color"] ="green"
                        start = start.append(start_data, ignore_index=True) 
                        current_price = row["close"]
                        trade_open = True
            else:
                ticker_data = {}   
                if self.get_pattern(row) == self.value:                             
                    ticker_data["x"] = index
                    ticker_data["Marker"] = row["close"]
                    ticker_data["Symbol"] ="circle"
                    ticker_data["Color"] ="gold"
                    pattern_data = pattern_data.append(ticker_data, ignore_index=True) 

                    current_data = data.loc[index-3:index]
                    trend, trend_value, reversal_value = helper.get_trends(current_data)
                    if trend == Trend.downtrend_reversal or trend == Trend.strong_downtrend_reversal:
                        start_data["x"] = index
                        start_data["Marker"] = row["close"]
                        start_data["Symbol"] ="star-triangle-up"
                        start_data["Color"] ="green"
                        start = start.append(start_data, ignore_index=True) 
                        current_price = row["close"]
                        trade_open = True
                    elif trend == Trend.downtrend or trend == Trend.downtrend_reversal or trend == Trend.strong_uptrend_reversal or trend == Trend.uptrend_reversal or trend ==Trend.sidetrend:
                        helper.index = 0
                        next_long_pattern = True


        CandleStick = go.Candlestick(x=data.index,
                           open=data.open,
                           high=data.high,
                           low=data.low,
                           close=data.close
                           )  
        Pattern = go.Scatter(x=pattern_data.x,
                           y=pattern_data.Marker,
                           mode='markers',
                           name ='markers',
                           marker=go.Marker(size=20,
                                            symbol=pattern_data["Symbol"],
                                            color=pattern_data["Color"]))

        StartData = go.Scatter(x=start.x,
                   y=start.Marker,
                   mode='markers',
                   name ='markers',
                   marker=go.Marker(size=20,
                                    symbol=start["Symbol"],
                                    color=start["Color"]))


        EndData = go.Scatter(x=end.x,
                       y=end.Marker,
                       mode='markers',
                       name ='markers',
                       marker=go.Marker(size=20,
                                        symbol=end["Symbol"],
                                        color=end["Color"]))

        iplot([CandleStick, Pattern,StartData,EndData])

In [91]:
class StatisticsManager:

    def __init__(self,name):    
        self.name =name  
        self.db = sqlite3.connect(self.name)
        

    def connectDB(self):
        self.db = sqlite3.connect(self.name)
        
    def get_trade_all(self):
        self.connectDB()
        query = '''SELECT * FROM Statistics'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_day(self, day):
        self.connectDB()
        query = '''SELECT close_time,current,close,number_of_shares,symbol,type,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by symbol,close_time having close_time = '''
        query += "'" + day + "'"
        query += ''' ORDER by returns'''
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data

    def get_trade_loss(self):
        self.connectDB()
        query = '''SELECT close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time having returns <= 0  ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_win(self):
        self.connectDB()
        query = '''SELECT close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time having returns > 0  ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    

    def get_trade_close_time(self):
        self.connectDB()
        query = '''SELECT close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_algorithm(self):
        self.connectDB()
        query = '''SELECT algorithm,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by algorithm ORDER by profit_ratio desc'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_symbol(self):
        self.connectDB()
        query = '''SELECT symbol,type,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by symbol,type ORDER by profit_ratio desc'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data


    def close(self):
        self.db.close()

    def deleteDB(self,db):
        if os.path.exists(db):
            os.remove(db)
        else:
            print("The file does not exist")

In [92]:
class PatternManager:
    
    def __init__(self,pattern):
         self.pattern = pattern
        
        
    def get_pattern(self,data):

            if self.pattern == PricePattern.BullishHammer:            
                return data[data['Hammer']!=0].dropna()

            if self.pattern == PricePattern.BullishHarami:           
                return data[data['Harami']!=0].dropna()

            if self.pattern == PricePattern.BullishInverseHammer:           
                return data[data['InverseHammer']!=0].dropna() 

            if self.pattern == PricePattern.BullishMorningStar:            
                return data[data['MorningStar']!=0].dropna()

            if self.pattern == PricePattern.BullishPiercingLine:           
                return data[data['PiercingLine']!=0].dropna()

            if self.pattern == PricePattern.BullishThreeWhiteSoldiers:            
                return data[data['WhiteSoldiers']!=0].dropna()

            if self.pattern == PricePattern.BullishEngulfing:            
                return data[data['Engulfing']!=0].dropna()

            if self.pattern == PricePattern.BearishDarkCloudCover:            
                return data[data['DarkCloudCover']!=0].dropna() 

            if self.pattern == PricePattern.BearishEveningStar:            
                return data[data['EveningStar']!=0].dropna() 

            if self.pattern == PricePattern.BearishHangingMan:            
                return data[data['HangingMan']!=0].dropna() 

            if self.pattern == PricePattern.BearishHarami:            
                return data[data['BearishHarami']!=0].dropna()  

            if self.pattern == PricePattern.BearishShootingStar:            
                return data[data['ShootingStar']!=0].dropna() 

            if self.pattern == PricePattern.BearishThreeBlackCrows:            
                return data[data['ThreeBlackCrows']!=0].dropna() 

            if self.pattern == PricePattern.BearishEngulfing:            
                return data[data['BearishEngulfing']!=0].dropna()

In [93]:
class LostWidget:
    def __init__(self,pattern=PricePattern.BullishHammer):
        self.pattern = pattern
        self.pattern_manager = PatternManager(pattern)
        self.graph = Graph(pattern=pattern)
        self.datalist = []
        self.statistic_manager = StatisticsManager(db)
        
        self.lost_count = widgets.Text(
                    value='47',
                    description='Days',
                    disabled=True) 
        
        self.lost_ticker = widgets.Text(
                    value='SBIN',
                    description='Ticker',
                    disabled=True) 
        
        self.lost_pattern_count = widgets.Text(
                       value='47',
                       description='Total Trade',
                        disabled=True)
        
        self.lost_current = widgets.Text(
                   value='47',
                   description='Current',
                    disabled=False)
        
        self.lost_current.on_submit(self.on_current_change)
        

        
        self.lost_date_menu = widgets.Dropdown(
                   options=['red', 'blue', 'green'],
                   value='red',
                   description='Date:')
        
        self.lost_date_menu.observe(self.on_date_change)
        
        
        self.lost_next_button = widgets.Button(description='Next')
        self.lost_next_button.on_click(self.on_next_button_clicked)
            
        self.lost_previous_button = widgets.Button(description='Previous')
        self.lost_previous_button.on_click(self.on_previous_button_clicked)
        
        self.lost_menu = widgets.HBox([self.lost_date_menu, self.lost_ticker, self.lost_count])        
        self.lost_pattern = widgets.HBox([ self.lost_pattern_count, self.lost_current])
        self.lost_button = widgets.HBox([ self.lost_previous_button, self.lost_next_button])
        self.lost_box = widgets.VBox([ self.lost_menu ,self.lost_pattern, self.lost_button])
        self.start()
        
    def on_date_change(self,change):
        if change['name'] == 'value' and (change['new'] != change['old']): 
            self.reset_tab()  
            
    def on_current_change(self,wdgt):    
        date = self.lost_date_menu.value 
        current = int(self.lost_current.value)
        self.show_graph(current)
    
    def on_next_button_clicked(self,b):
        date = self.lost_date_menu.value 
        current = int(self.lost_current.value)
        self.show_graph(current+1)
        
    def on_previous_button_clicked(self,b):
        date = self.lost_date_menu.value 
        current = int(self.lost_current.value)
        self.show_graph(current-1)
        
    def get_data(self,date):
        df = pd.DataFrame(columns = ['open', 'high', 'low','close','volume'])
        for ticker in tickerList:
            filename = data_directory+ticker+".csv"
            data= pd.read_csv(filename)
            data= data.drop('Unnamed: 0',axis=1)
            data = data[data['date'] == date]
            df = df.append(data, ignore_index=True)
        return df
    
    
    def pattern_data(self,data):
        pattern = self.pattern_manager.get_pattern(data)
        data_list =[]
        for index, row in pattern.iterrows():
            data_list.append(data.loc[index-4:index+14])
        self.datalist = data_list 
        
    def show_graph(self,current):
        if current >= len(self.datalist):
            current = len(self.datalist)
        if current < 1:
            current =  1        
        self.lost_current.value = str(current)  
        self.lost_ticker.value = self.datalist[current-1]['ticker'].values[0]
        self.graph.show(self.datalist[current-1],self.lost_ticker.value)
        
    def reset_tab(self):
        date = self.lost_date_menu.value    
        self.pattern_data(self.get_data(date))
        self.lost_pattern_count.value = str(len(self.datalist))
        self.lost_current.value = str(1)        
        self.show_graph(1)    
    
    def start(self):
        date_list,trade_data = self.get_all_loss()
        self.lost_date_menu.options = date_list
        self.lost_count.value = str(len(date_list))        
        
        
    def get_all_loss(self):     
        loss_data = self.statistic_manager.get_trade_loss()
        trade_data = pd.DataFrame(columns = ['close_time', 'loss', 'profit','open','loss_ratio','profit_ratio','returns'])
        date_list = loss_data["close_time"].unique().tolist()
        for day in date_list:
            row_data = self.statistic_manager.get_trade_day(day)
            trade_data = trade_data.append(row_data, ignore_index=True)
        return date_list,trade_data
    
    

In [94]:
class WinWidget:
    def __init__(self,pattern=PricePattern.BullishHammer):
        self.pattern = pattern
        self.pattern_manager = PatternManager(pattern)
        self.graph = Graph(pattern=pattern)
        self.datalist = []
        self.statistic_manager = StatisticsManager(db)
        self.win_count = widgets.Text(
                    value='47',
                    description='Days',
                    disabled=True)      
        
        self.win_pattern_count = widgets.Text(
                       value='47',
                       description='Total Trade',
                        disabled=True)
        self.win_ticker = widgets.Text(
                    value='SBIN',
                    description='Ticker',
                    disabled=True) 
        
        self.win_current = widgets.Text(
                   value='47',
                   description='Current',
                    disabled=False)
        
        self.win_current.on_submit(self.on_current_change)
        
       
        
        self.win_date_menu = widgets.Dropdown(
                   options=['red', 'blue', 'green'],
                   value='red',
                   description='Date:')
        
        self.win_date_menu.observe(self.on_date_change)
        
        self.win_next_button = widgets.Button(description='Next')
        self.win_next_button.on_click(self.on_next_button_clicked)
        
        self.win_previous_button = widgets.Button(description='Previous')
        self.win_previous_button.on_click(self.on_previous_button_clicked)
        
        self.win_menu = widgets.HBox([ self.win_date_menu ,  self.win_ticker,self.win_count])        
        self.win_pattern = widgets.HBox([self.win_pattern_count,  self.win_current])
        self.win_button = widgets.HBox([self.win_previous_button,  self.win_next_button])
        self.win_box = widgets.VBox([self.win_menu,self.win_pattern,self.win_button])
        self.start()
        
    def on_date_change(self,change):
        if change['name'] == 'value' and (change['new'] != change['old']): 
            self.reset_tab()  
        
    def on_ticker_change(self,change):
        if change['name'] == 'value' and (change['new'] != change['old']): 
            self.reset_tab()
        
    def on_current_change(self,wdgt):    
        date = self.win_date_menu.value
        current = int(self.win_current.value)
        self.show_graph(current)
    
    def on_next_button_clicked(self,b):
        date = self.win_date_menu.value
        current = int(self.win_current.value)
        self.show_graph(current+1)
        
    def on_previous_button_clicked(self,b):
        date = self.win_date_menu.value
        current = int(self.win_current.value)
        self.show_graph(current-1)
        
    def get_data(self,date):
        df = pd.DataFrame(columns = ['open', 'high', 'low','close','volume'])
        for ticker in tickerList:
            filename = data_directory+ticker+".csv"
            data= pd.read_csv(filename)
            data= data.drop('Unnamed: 0',axis=1)
            data = data[data['date'] == date]
            df = df.append(data, ignore_index=True)
        return df
    
    def pattern_data(self,data):
        pattern = self.pattern_manager.get_pattern(data)
        data_list =[]
        for index, row in pattern.iterrows():
            data_list.append(data.loc[index-4:index+14])
        self.datalist = data_list 
        
    def show_graph(self,current):
        if current >= len(self.datalist):
            current = len(self.datalist)
        if current < 1:
            current =  1        
        self.win_current.value = str(current) 
        self.win_ticker.value = self.datalist[current-1]['ticker'].values[0]
        self.graph.show(self.datalist[current-1], self.win_ticker.value)
        
    def reset_tab(self):
        date = self.win_date_menu.value
        self.pattern_data(self.get_data(date))
        self.win_pattern_count.value = str(len(self.datalist))
        self.win_current.value = str(1)        
        self.show_graph(1)   
    
    def start(self):
        date_list,trade_data = self.get_all_win()
        self.win_date_menu.options = date_list
        self.win_count.value = str(len(date_list))        
        
        
    def get_all_win(self):        
        loss_data = self.statistic_manager.get_trade_win()
        trade_data = pd.DataFrame(columns = ['close_time', 'loss', 'profit','open','loss_ratio','profit_ratio','returns'])
        date_list = loss_data["close_time"].unique().tolist()
        for day in date_list:
            row_data = self.statistic_manager.get_trade_day(day)
            trade_data = trade_data.append(row_data, ignore_index=True)
        return date_list,trade_data

In [95]:
class AllWidget:
    def __init__(self,pattern=PricePattern.BullishHammer):
        self.pattern = pattern
        self.pattern_manager = PatternManager(pattern)
        self.graph = Graph(pattern=pattern)
        self.datalist = []
        self.statistic_manager = StatisticsManager(db)

        self.days = widgets.Text(
            value='47',
            description='Days',
            disabled=True)

        self.date = widgets.Text(
            value='2021-04-30-19:19:32',
            description='date',
            disabled=True)

        self.trade = widgets.Text(
            value='47',
            description='Total Trade',
            disabled=True)

        self.current = widgets.Text(
            value='47',
            description='Current',
            disabled=False)

        self.current.on_submit(self.on_current_change)

        self.ticker_menu = widgets.Dropdown(
            options=tickerList,
            value=tickerList[0],
            description='Date:')

        self.ticker_menu.observe(self.on_ticker_change)

        self.next_button = widgets.Button(description='Next')
        self.next_button.on_click(self.on_next_button_clicked)

        self.previous_button = widgets.Button(description='Previous')
        self.previous_button.on_click(self.on_previous_button_clicked)

        self.menu = widgets.HBox([self.ticker_menu, self.date, self.days])
        self.trade = widgets.HBox([self.trade, self.current])
        self.button = widgets.HBox([self.previous_button, self.next_button])
        self.all_box = widgets.VBox([self.menu, self.trade, self.button]) 
        self.reset_tab()

    def on_ticker_change(self, change):
        if change['name'] == 'value' and (change['new'] != change['old']):
            self.reset_tab()

    def on_current_change(self, wdgt):
        date = self.ticker_menu.value
        current = int(self.current.value)
        self.show_graph(current)

    def on_next_button_clicked(self, b):
        date = self.ticker_menu.value
        current = int(self.current.value)
        self.show_graph(current + 1)

    def on_previous_button_clicked(self, b):
        date = self.ticker_menu.value
        current = int(self.current.value)
        self.show_graph(current - 1)

    def get_data(self, ticker):
        filename = data_directory + ticker + ".csv"
        data = pd.read_csv(filename)
        data = data.drop('Unnamed: 0', axis=1)
        return data

    def pattern_data(self, data):
        pattern = self.pattern_manager.get_pattern(data)
        data_list = []
        for index, row in pattern.iterrows():
            data_list.append(data.loc[index - 4:index + 14])
        self.datalist = data_list

    def show_graph(self, current):
        if current >= len(self.datalist):
            current = len(self.datalist)
        if current < 1:
            current = 1
        self.current.value = str(current)
        self.date.value = self.datalist[current - 1]['date'].values[0]
        self.graph.show(self.datalist[current - 1], self.date.value)

    def reset_tab(self):
        ticker = self.ticker_menu.value
        data = self.get_data(ticker)
        self.days.value = str(len(data['date'].unique()))
        self.pattern_data(data)
        self.trade.value = str(len(self.datalist))
        self.current.value = str(1)
        self.show_graph(1)

In [96]:
class TabWidget:
    
    def __init__(self,pattern=PricePattern.BullishHammer):
        self.win_widget = WinWidget(pattern=pattern)
        self.lost_widget = LostWidget(pattern=pattern) 
        self.all_widget = AllWidget(pattern=pattern) 
        self.tab = widgets.Tab()
        
        
    def show_tab(self):        
        children = [self.all_widget.all_box,self.lost_widget.lost_box, self.win_widget.win_box]        
        self.tab.children = children
        self.tab.set_title(0, 'Ticker Analysis')
        self.tab.set_title(1, 'Loss Analysis')
        self.tab.set_title(2, 'Win Analysis')         
        display.clear_output(wait=True)   
        return self.tab   

    
    

In [105]:
def showTable(data,index=False):
    print(tabulate(data, headers='keys', tablefmt='psql',showindex=index))

In [97]:
hammer_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\hammer\\statistics.db"
hammer_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\hammer\\"

bullish_engulfing_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\bullish_engulfing\\statistics.db"
bullish_engulfing_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\bullish_engulfing\\"

bullish_harami_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\bullish_harami\\statistics.db"
bullish_harami_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\bullish_harami\\"

inverse_hammer_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\inverse_hammer\\statistics.db"
inverse_hammer_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\inverse_hammer\\"

morning_star_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\morning_star\\statistics.db"
morning_star_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\morning_star\\"

piercing_line_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\piercing_line\\statistics.db"
piercing_line_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\piercing_line\\"

white_soldiers_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\white_soldiers\\statistics.db"
white_soldiers_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\white_soldiers\\"

db_list = [hammer_db,
           bullish_engulfing_db,
           bullish_harami_db,
           inverse_hammer_db,
           morning_star_db,
           piercing_line_db,
           white_soldiers_db
          ]
data_directory_list=[hammer_directory,
                     bullish_engulfing_directory,
                     bullish_harami_directory,
                     inverse_hammer_directory,
                     morning_star_directory,
                     piercing_line_directory,
                     white_soldiers_directory
                    ]
pattern_list=[PricePattern.BullishHammer,
              PricePattern.BullishEngulfing,
              PricePattern.BullishHarami,
              PricePattern.BullishInverseHammer,
              PricePattern.BullishMorningStar,
              PricePattern.BullishPiercingLine,
               PricePattern.BullishThreeWhiteSoldiers
             ]
index = 0
db = db_list[index]
data_directory = data_directory_list[index]
pattern=pattern_list[index]

In [98]:
TabWidget(pattern).show_tab()

In [106]:
trade_data = pd.DataFrame(columns = ['algorithm', 'loss', 'profit','open','loss_ratio','profit_ratio'])
manager = StatisticsManager(hammer_db)
for db in db_list:
    manager.name = db
    row_data = manager.get_trade_algorithm()
    trade_data = trade_data.append(row_data, ignore_index=True)
trade_data
showTable(trade_data)

+---------------------------+--------+----------+--------+--------------+----------------+-----------+
| algorithm                 |   loss |   profit |   open |   loss_ratio |   profit_ratio |   returns |
|---------------------------+--------+----------+--------+--------------+----------------+-----------|
| BullishHammer             |    308 |      293 |      0 |         0.51 |           0.49 |  20504    |
| BullishEngulfing          |    313 |      253 |      0 |         0.55 |           0.45 |  18869.3  |
| BullishHarami             |    316 |      286 |      0 |         0.52 |           0.48 |  17296    |
| BullishInverseHammer      |    303 |      275 |      0 |         0.52 |           0.48 |  -1939.55 |
| BullishMorningStar        |    175 |      112 |      0 |         0.61 |           0.39 | -10389.8  |
| BullishPiercingLine       |     97 |       46 |      0 |         0.68 |           0.32 |  -7637.35 |
| BullishThreeWhiteSoldiers |     95 |       89 |      0 |         0.52 |

['2021-04-30-08:50:08',
 '2021-05-20-08:50:08',
 '2021-05-21-08:50:08',
 '2021-05-10-08:50:08',
 '2021-05-19-08:50:08',
 '2021-05-28-08:50:08',
 '2021-05-25-08:50:08',
 '2021-05-04-08:50:08',
 '2021-05-09-08:50:08',
 '2021-05-26-08:50:08',
 '2021-04-29-08:50:08',
 '2021-05-08-08:50:08',
 '2021-05-31-08:50:08',
 '2021-05-11-08:50:08',
 '2021-05-13-08:50:08',
 '2021-05-05-08:50:08',
 '2021-05-15-08:50:08',
 '2021-05-18-08:50:08',
 '2021-05-30-08:50:08',
 '2021-05-24-08:50:08']